# Make tables for CIS-PD curation paper

In [1]:
# Importing the Libraries
import os
import platform
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import pathlib
import pickle
import time
import re
import copy
%matplotlib inline

In [11]:
path = r'Y:\CIS-PD MUSC\decoded_forms'
data124 = pd.read_hdf(os.path.join(path,'updrs_124.h5'))
data3 = pd.read_hdf(os.path.join(path,'updrs_part3.h5'))

In [12]:
data124

,Subject ID,SiteID,Visit,FormDate,Primary source of information,1.1,1.2,1.3,1.4,1.5,...,2.11,2.12,2.13,4.1,4.2,4.3,4.4,4.5,4.6,DataCollected
0,1000.0,1313.0,Baseline,2017-06-15,Patient,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,Yes
1,1000.0,1313.0,2 Weeks,2017-06-29,Patient,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,Yes
2,1000.0,1313.0,1 Month,2017-07-13,Patient,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,Yes
3,1000.0,1313.0,3 Months,2017-09-21,Patient,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,Yes
4,1000.0,1313.0,6 Months,2017-12-11,Patient,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,Yes
5,1001.0,1313.0,Baseline,2017-06-15,Patient,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes
6,1001.0,1313.0,2 Weeks,2017-06-29,Patient,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes
7,1001.0,1313.0,1 Month,2017-07-14,Patient,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes
8,1001.0,1313.0,3 Months,2017-09-21,Patient,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,Yes
9,1001.0,1313.0,6 Months,2017-12-21,Patient,0.0,0.0,1.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,Yes


In [14]:
data124.columns.values

array(['Subject ID', 'SiteID', 'Visit', 'FormDate',
       'Primary source of information', '1.1', '1.2', '1.3', '1.4', '1.5',
       '1.6', 'Who is filling out this questionnaire', '1.7', '1.8',
       '1.9', '1.10', '1.11', '1.12', '1.13', '2.1', '2.2', '2.3', '2.4',
       '2.5', '2.6', '2.7', '2.8', '2.9', '2.10', '2.11', '2.12', '2.13',
       '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', 'DataCollected'],
      dtype=object)

# Create Table for parts 1, 2, and 4

In [95]:
data124.columns.values

array(['Subject ID', 'SiteID', 'Visit', 'FormDate',
       'Primary source of information', '1.1', '1.2', '1.3', '1.4', '1.5',
       '1.6', 'Who is filling out this questionnaire', '1.7', '1.8',
       '1.9', '1.10', '1.11', '1.12', '1.13', '2.1', '2.2', '2.3', '2.4',
       '2.5', '2.6', '2.7', '2.8', '2.9', '2.10', '2.11', '2.12', '2.13',
       '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', 'DataCollected'],
      dtype=object)

In [100]:
list = datadict.loc[datadict.FormID == 238.0]['FieldNm'].tolist()

In [118]:
# slice list for parts 1,2,4, subtract part 3
qlist = list[:30] + list[70:-1]

In [ ]:
qlist

In [122]:
tabledata124 = {
'Question': ['Subject ID', 'SiteID', 'Visit',
             'Date of assessment',
             'Primary source of information',
             '1.1 Cognitive Impairment',
             '1.2 Hallucinations and Psychosis',
             '1.3 Depressed Mood',
             '1.4 Anxious Mood',
             '1.5 Apathy',
             '1.6 Features of Dopamine Dysregulation Syndrome',
             'Who is filling out this questionnaire',
             '1.7 Sleep Problems',
             '1.8 Daytime Sleepiness',
             '1.9 Pain and Other Sensations',
             '1.10 Urinary Problems',
             '1.11 Constipation Problems',
             '1.12 Light Headedness on Standing',
             '1.13 Fatigue',
             '2.1 Speech',
             '2.2 Saliva & Drooling',
             '2.3 Chewing and Swallowing',
             '2.4 Eating Tasks',
             '2.5 Dressing',
             '2.6 Hygiene',
             '2.7 Handwriting',
             '2.8 Doing Hobbies and Other Activities',
             '2.9 Turning in Bed',
             '2.10 Tremor',
             '2.11 Getting Out of Bed, a Car, or a Deep Chair',
             '2.12 Walking and Balance',
             '2.13 Freezing',
             '4.1 Time Spent With Dyskinesias',
             '4.2 Functional Impact of Dyskinesias',
             '4.3 Time Spent in the Off State',
             '4.4 Functional Impact of Fluctuations',
             '4.5 Complexity of Motor Fluctuations',
             '4.6 Painful Off-State Dystonia',
             'Data Collected'],
'Variable Name': data124.columns.values.tolist(),
'Variable details': ['Subject ID', 'SiteID', 'Visit', 'FormDate',
                     'One of: (\'Patient\', \'Caregiver\', \'Patient and Caregiver in Equal Proportion\')', 
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Patient\', \'Caregiver\', \'Patient and Caregiver in Equal Proportion\')', 
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'no\',\'yes\')']
}
df124 = pd.DataFrame(tabledata124)
# save as csv file to create table

In [123]:
# save table as csv file
tablepath = r'Y:\CIS-PD MUSC\tables'
filename = os.path.join(tablepath, 'tablepart124.csv')
with open(filename,'wb') as f:
    df124.to_csv(filename, sep=',')

# Create Table for part 3
Notes:
why does data3 have 
- ParticipantState and 3b - same
- What's the difference b/n UTC and FormTime?


In [94]:
data3.columns.values#.tolist()

array(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'UTC', 'FormTime',
       'ParticipantState', 'Hours since last PD medication dose', '3a',
       '3b', '3c', '3C1', '3.1', '3.2', '3.3 Neck',
       '3.3 Right Upper Extremity', '3.3 Left Upper Extremity',
       '3.3 Right Lower Extremity', '3.3 Left Lower Extremity',
       '3.4 Right Hand', '3.4 Left Hand', '3.5 Right Hand',
       '3.5 Left Hand', '3.6 Right Hand', '3.6 Left Hand',
       '3.7 Right Foot', '3.7 Left Foot', '3.8 Right Leg', '3.8 Left Leg',
       '3.9', '3.10', '3.11', '3.12', '3.13', '3.14', '3.15 Right Hand',
       '3.15 Left Hand', '3.16 Right Hand', '3.16 Left Hand',
       '3.17 Right Upper Extremity', '3.17 Left Upper Extremity',
       '3.17 Right Lower Extremity', '3.17 Left Lower Extremity',
       '3.17 Lip-Jaw', '3.18', '3.19A', '3.19B', '3.20', 'DataCollected'],
      dtype=object)

In [87]:
tabledata = {
'Question': ['Subject ID', 'SiteID', 'Visit', 'Date of assessment',
             'Time of assessment', 'FormTime',
             # repeated items with 3b - 'ParticipantState', 'Hours since last PD medication dose'???
             '3b Mark the patient\'s clinical state',
             'Hours since last PD medication dose',
             # 3a, 3b, 3c, 3C1
             '3a Is the patient on medication for treating the symptoms of Parkinson\'s Disease?',
             '3b Mark the patient\'s clinical state', 
             '3c Is the patient on Levodopa?',
             '3.C1 Minutes since last levodopa dose',
             '3.1 Speech',
             '3.2 Facial Expression',
             '3.3 Rigidity - Neck',
             '3.3 Rigidity - Right Upper Extremity',
             '3.3 Rigidity - Left Upper Extremity',
             '3.3 Rigidity - Right Lower Extremity',
             '3.3 Rigidity - Left Lower Extremity',
             '3.4 Finger Tapping - Right Hand',
             '3.4 Finger Tapping - Left Hand',
             '3.5 Hand Movements - Right Hand',
             '3.5 Hand Movements - Left Hand',
             '3.6 Pronation-Supination Movements of Hands - Right Hand',
             '3.6 Pronation-Supination Movements of Hands - Left Hand',
             '3.7 Toe Tapping - Right Foot',
             '3.7 Toe Tapping - Left Foot',
             '3.8 Leg Agility - Right Leg',
             '3.8 Leg Agility - Left Leg',
             '3.9 Arising From Chair',
             '3.10 Gait',
             '3.11 Freezing of Gait',
             '3.12 Postural Stability',
             '3.13 Posture',
             '3.14 Global Spontaneity of Movement Body Bradykinesia',
             '3.15 Postural Tremor of the Hands - Right Hand',
             '3.15 Postural Tremor of the Hands - Left Hand',
             '3.16 Kinetic Tremor of the Hands - Right Hand',
             '3.16 Kinetic Tremor of the Hands - Left Hand',
             '3.17 Rest Tremor Amplitude - Right Upper Extremity',
             '3.17 Rest Tremor Amplitude - Left Upper Extremity',
             '3.17 Rest Tremor Amplitude - Right Lower Extremity',
             '3.17 Rest Tremor Amplitude - Left Lower Extremity',
             '3.17 Rest Tremor Amplitude - Lip/Jaw',
             '3.18 Constancy of Rest Tremor',
             '3.19 A Were dyskinesias (chorea or dystonia) present during examination?',
             '3.19 B Did these movements interfere with your ratings?',
             '3.20 Hoen and Yahr Stage',
             'Data collected'],
'Variable Name': data3.columns.values.tolist(),
'Variable details': ['Subject ID', 'SiteID', 'Visit', 'Date of assessment',
                     'Datetime', 'FormTime',
                     # repeated items with 3b - 'ParticipantState', 'Hours since last PD medication dose'???
                     'One of: (\'on\',\'off\')',
                     'Integer',
                     # 3a, 3b, 3c, 3C1
                     'One of: (\'no\',\'yes\')',
                     'One of: (\'on\',\'off\')', 
                     'One of: (\'no\',\'yes\')',
                     'Integer',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'no\',\'yes\')',
                     'One of: (\'no\',\'yes\')',
                     'One of: (0, 1, 2, 3, 4, 5)',
                     'One of: (\'no\',\'yes\')']
}
df = pd.DataFrame(tabledata)
# save as csv file to create table

In [ ]:
df

In [90]:
# save table as csv file
tablepath = r'Y:\CIS-PD MUSC\tables'
filename = os.path.join(tablepath, 'tablepart3.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

# Check data dictionary

In [24]:
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\codelist.sas7bdat'
codelist = pd.read_sas(filename)
codelist.columns = codelist.columns.str.replace('z', '')
codelist = codelist.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\datadictionary.sas7bdat'
datadict = pd.read_sas(filename, format = 'sas7bdat', encoding='iso-8859-1')
datadict.columns = datadict.columns.str.replace('z', '')

In [17]:
codelist.head()

,GroupID,ItemNb,ItemNm
0,1.0,1.0,"Global non-project related tables, such as zCo..."
1,1.0,2.0,"Global project related study design tables, su..."
2,1.0,3.0,"Global project related form setup tables, such..."
3,1.0,4.0,"Subject related No CRF tables, such as subject..."
4,1.0,5.0,"CRF related no CRF tables, such as subject CRF..."


In [29]:
datadict.head()

,FormID,CRFNb,FieldNb,FieldNm,SASNm,SASLabel,FieldTypeNm,Length,Unit,Required,CodeGroup
0,101.0,101,zConsentDate,Date of signed informed consent,NaN,NaN,Date,0.0,NaN,0.0,NaN
1,101.0,101,zAge,Age,NaN,NaN,Number,0.0,NaN,0.0,NaN
2,101.0,101,Q03,Motor fluctuations believed to be on average 2...,NaN,NaN,Radio button,0.0,NaN,2.0,51.0
3,101.0,101,Q04,Diagnosis of idiopathic Parkinsons disease,NaN,NaN,Radio button,0.0,NaN,0.0,51.0
4,101.0,101,Q05,Hoehn & Yahr stage 1-3,NaN,NaN,Radio button,0.0,NaN,0.0,51.0


In [91]:
# codelist doesn't have form 508 or 238, only datadict does

# codelist.loc[codelist.GroupID == 508.0]
codelist.loc[codelist.GroupID == 1.0].head(3)


,GroupID,ItemNb,ItemNm
0,1.0,1.0,"Global non-project related tables, such as zCo..."
1,1.0,2.0,"Global project related study design tables, su..."
2,1.0,3.0,"Global project related form setup tables, such..."


In [ ]:
codelist.groupby('GroupID').groups

In [19]:
datadict.loc[datadict.FormID == 238.0]

,FormID,CRFNb,FieldNb,FieldNm,SASNm,SASLabel,FieldTypeNm,Length,Unit,Required,CodeGroup
24,238.0,238,zDataCollected,Data Collected,NaN,NaN,Radio button,0.0,NaN,1.0,51.0
25,238.0,238,zFormDate,Date of assessment,NaN,NaN,Date,0.0,NaN,1.0,NaN
26,238.0,238,Q01,Primary source of information,NaN,NaN,Radio button,0.0,NaN,2.0,580.0
27,238.0,238,Q02,1.1 Cognitive Impairment,NaN,NaN,Number,0.0,(0-4),2.0,NaN
28,238.0,238,Q03,1.2 Hallucinations and Psychosis,NaN,NaN,Number,0.0,(0-4),2.0,NaN
29,238.0,238,Q04,1.3 Depressed Mood,NaN,NaN,Number,0.0,(0-4),2.0,NaN
30,238.0,238,Q05,1.4 Anxious Mood,NaN,NaN,Number,0.0,(0-4),2.0,NaN
31,238.0,238,Q06,1.5 Apathy,NaN,NaN,Number,0.0,(0-4),2.0,NaN
32,238.0,238,Q07,1.6 Features of Dopamine Dysregulation Syndrome,NaN,NaN,Number,0.0,(0-4),2.0,NaN
33,238.0,238,Q08,Who is filling out this questionnaire,NaN,NaN,Radio button,0.0,NaN,2.0,580.0


In [25]:
datadict.loc[datadict.FormID == 101.0]

,FormID,CRFNb,FieldNb,FieldNm,SASNm,SASLabel,FieldTypeNm,Length,Unit,Required,CodeGroup
0,101.0,101,zConsentDate,Date of signed informed consent,NaN,NaN,Date,0.0,NaN,0.0,NaN
1,101.0,101,zAge,Age,NaN,NaN,Number,0.0,NaN,0.0,NaN
2,101.0,101,Q03,Motor fluctuations believed to be on average 2...,NaN,NaN,Radio button,0.0,NaN,2.0,51.0
3,101.0,101,Q04,Diagnosis of idiopathic Parkinsons disease,NaN,NaN,Radio button,0.0,NaN,0.0,51.0
4,101.0,101,Q05,Hoehn & Yahr stage 1-3,NaN,NaN,Radio button,0.0,NaN,0.0,51.0
5,101.0,101,Q06,Ability to travel to a study site to attend th...,NaN,NaN,Radio button,0.0,NaN,0.0,51.0
6,101.0,101,Q07,Ability to read and understand English,NaN,NaN,Radio button,0.0,NaN,0.0,51.0
7,101.0,101,Q08,Owns and is the sole user of an iPhone 5s and ...,NaN,NaN,Radio button,0.0,NaN,0.0,51.0
8,101.0,101,Q11,iPhone model to be used by subject,NaN,NaN,Text,50.0,NaN,4.0,NaN
9,101.0,101,Q12,iOS version to be used,NaN,NaN,Text,50.0,NaN,4.0,NaN


In [42]:
datadict.groupby('FormID').groups

{101.0: Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64'),
 126.0: Int64Index([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], dtype='int64'),
 238.0: Int64Index([ 24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,
              37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,
              50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
              63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,
              76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,
              89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100],
            dtype='int64'),
 239.0: Int64Index([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
             114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
             127],
            dtype='int64'),
 240.0: Int64Index([128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
             141, 142, 143, 144, 145],
       

In [20]:
datadict.loc[datadict.FormID == 508.0]

,FormID,CRFNb,FieldNb,FieldNm,SASNm,SASLabel,FieldTypeNm,Length,Unit,Required,CodeGroup
258,508.0,508,zDataCollected,Data Collected,NaN,NaN,Radio button,0.0,NaN,1.0,51.0
259,508.0,508,zFormDate,Date of assessment,NaN,NaN,Date,0.0,NaN,1.0,NaN
260,508.0,508,zFormTime,Time of assessment,NaN,NaN,Date and/or Time,0.0,NaN,4.0,NaN
261,508.0,508,Q01,What is the participants state,NaN,NaN,Radio button,0.0,NaN,0.0,581.0
262,508.0,508,Q02,Hours since last Parkinsons Disease medicatio...,NaN,NaN,Number,0.0,hours,4.0,NaN
263,508.0,508,Q33,3.1 Speech,NaN,NaN,Number,0.0,(0-4),2.0,NaN
264,508.0,508,Q34,3.2 Facial Expression,NaN,NaN,Number,0.0,(0-4),2.0,NaN
265,508.0,508,Q35,3.3 Rigidity - Neck,NaN,NaN,Number,0.0,(0-4),2.0,NaN
266,508.0,508,Q36,3.3 Rigidity - Right Upper Extremity,NaN,NaN,Number,0.0,(0-4),2.0,NaN
267,508.0,508,Q37,3.3 Rigidity - Left Upper Extremity,NaN,NaN,Number,0.0,(0-4),2.0,NaN


# Create Demographics table

In [4]:
path = r'Y:\CIS-PD MUSC\decoded_forms'

In [8]:
# items: gender, race, ethnicity, age
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\subjenrollment.sas7bdat'
subjenroll = pd.read_sas(filename, format = 'sas7bdat', encoding='iso-8859-1')
subjenroll.columns = subjenroll.columns.str.replace('z', '')
print(subjenroll.head(5))

   SubjectCode FoxInsightID  SiteID  ConsentBeforeEnrollment  Gender  Race  \
0       1000.0     cisuaba1  1313.0                      1.0     1.0   5.0   
1       1001.0     cisuabb2  1313.0                      1.0     1.0   5.0   
2       1002.0     cisuabc3  1313.0                      1.0     1.0   5.0   
3       1003.0     cisuabd4  1313.0                      1.0     2.0   3.0   
4       1004.0      cisnwa1  1332.0                      1.0     1.0   5.0   

   Ethnicity   Age  ConsentDt EnrollmentDt  
0        2.0  63.0 2017-06-15   2017-06-15  
1        2.0  64.0 2017-06-15   2017-06-15  
2        2.0  51.0 2017-06-16   2017-06-16  
3        2.0  60.0 2017-06-19   2017-06-19  
4        2.0  52.0 2017-06-21   2017-06-21  


In [15]:
tabledata = {
'Question': ['Age', 'Gender', 'Race', 'Ethnicity', 'Hoehn & Yahr stage 1-3'],
'Variable Name': ['Age', 'Gender', 'Race', 'Ethnicity', 'Hoehn & Yahr stage 1-3'],
'Variable details': ['Integer',
                     'One of: (\'Male\',\'Female\')',
                     'One of: (\'American Indian or Alaska Native\',\'Asian\',\'Black or Afican American\',\'Native Hawaiian or Other Pacific Islander\',\'White\',\'Unknown\')',
                     'One of: (\'Hispanic or Latino\',\'Not Hispanic or Latino\',\'Unknown\')',
                     'One of: (\'no\',\'yes\')']
}
df = pd.DataFrame(tabledata)

# save table as csv file
tablepath = r'Y:\CIS-PD MUSC\tables'
filename = os.path.join(tablepath, 'tabledemo.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

In [13]:
df

,Question,Variable Name,Variable details
0,Age,Age,Integer
1,Gender,Gender,"One of: ('Male','Female')"
2,Race,Race,"One of: ('American Indian or Alaska Native','A..."
3,Ethnicity,Ethnicity,"One of: ('Hispanic or Latino','Not Hispanic or..."
4,Hoehn & Yahr stage 1-3,Hoehn & Yahr stage 1-3,"One of: ('no','yes')"


In [5]:
data101 = pd.read_hdf(os.path.join(path,'form101.h5'))
print(data101.head(5))

   SubjectCode  SiteID   VisitNm    Q11     Q12 ConsentDate   Age  Q03  Q04  \
0       1000.0  1313.0  Baseline  A1784  10.3.2  2017-06-15  63.0  Yes  Yes   
1       1001.0  1313.0  Baseline  A1778  10.3.2  2017-06-15  64.0   No  Yes   
2       1002.0  1313.0  Baseline  A1586  10.3.2  2017-06-16  51.0   No  Yes   
3       1003.0  1313.0  Baseline  A1522  10.3.2  2017-06-19  60.0  Yes  Yes   
4       1004.0  1332.0  Baseline  A1660  10.3.2  2017-06-21  52.0  Yes  Yes   

   Q05  Q06  Q07  Q08 Q09 Q10  
0  Yes  Yes  Yes  Yes  No  No  
1  Yes  Yes  Yes  Yes  No  No  
2  Yes  Yes  Yes  Yes  No  No  
3  Yes  Yes  Yes  Yes  No  No  
4  Yes  Yes  Yes  Yes  No  No  


In [6]:
data126 = pd.read_hdf(os.path.join(path,'form126.h5'))
print(data101.head(5))

   SubjectCode  SiteID   VisitNm    Q11     Q12 ConsentDate   Age  Q03  Q04  \
0       1000.0  1313.0  Baseline  A1784  10.3.2  2017-06-15  63.0  Yes  Yes   
1       1001.0  1313.0  Baseline  A1778  10.3.2  2017-06-15  64.0   No  Yes   
2       1002.0  1313.0  Baseline  A1586  10.3.2  2017-06-16  51.0   No  Yes   
3       1003.0  1313.0  Baseline  A1522  10.3.2  2017-06-19  60.0  Yes  Yes   
4       1004.0  1332.0  Baseline  A1660  10.3.2  2017-06-21  52.0  Yes  Yes   

   Q05  Q06  Q07  Q08 Q09 Q10  
0  Yes  Yes  Yes  Yes  No  No  
1  Yes  Yes  Yes  Yes  No  No  
2  Yes  Yes  Yes  Yes  No  No  
3  Yes  Yes  Yes  Yes  No  No  
4  Yes  Yes  Yes  Yes  No  No  


In [ ]:
data101 = pd.read_hdf(os.path.join(path,'form238.h5'))
print(data101.head(5))

In [ ]:
data101 = pd.read_hdf(os.path.join(path,'form101.h5'))
print(data101.head(5))

# Create Apple watch and phone table
- form101
    - Q11 iphone model
    - Q12 iOS version

In [4]:
path = r'Y:\CIS-PD MUSC\decoded_forms'

In [26]:
tabledata = {
'Question': ['iPhone model to be used by subject', 'iOS version to be used'],
'Variable Name': ['iphone', 'ios'],
'Variable details': ['Text',
                     'Text']
}
df = pd.DataFrame(tabledata)

# save table as csv file
tablepath = r'Y:\CIS-PD MUSC\tables'
filename = os.path.join(tablepath, 'tablespecs.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

### finding questions on forms for Apple watch and phone info

In [21]:
# form101 Q11 and 12 have phone model and iOS version
path = r'Y:\CIS-PD MUSC\decoded_forms'
data101 = pd.read_hdf(os.path.join(path,'form101.h5'))

In [23]:
data101.head(3)

,SubjectCode,SiteID,VisitNm,Q11,Q12,ConsentDate,Age,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10
0,1000.0,1313.0,Baseline,A1784,10.3.2,2017-06-15,63.0,Yes,Yes,Yes,Yes,Yes,Yes,No,No
1,1001.0,1313.0,Baseline,A1778,10.3.2,2017-06-15,64.0,No,Yes,Yes,Yes,Yes,Yes,No,No
2,1002.0,1313.0,Baseline,A1586,10.3.2,2017-06-16,51.0,No,Yes,Yes,Yes,Yes,Yes,No,No


In [8]:
# items: gender, race, ethnicity, age
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\subjenrollment.sas7bdat'
subjenroll = pd.read_sas(filename, format = 'sas7bdat', encoding='iso-8859-1')
subjenroll.columns = subjenroll.columns.str.replace('z', '')
print(subjenroll.head(5))

   SubjectCode FoxInsightID  SiteID  ConsentBeforeEnrollment  Gender  Race  \
0       1000.0     cisuaba1  1313.0                      1.0     1.0   5.0   
1       1001.0     cisuabb2  1313.0                      1.0     1.0   5.0   
2       1002.0     cisuabc3  1313.0                      1.0     1.0   5.0   
3       1003.0     cisuabd4  1313.0                      1.0     2.0   3.0   
4       1004.0      cisnwa1  1332.0                      1.0     1.0   5.0   

   Ethnicity   Age  ConsentDt EnrollmentDt  
0        2.0  63.0 2017-06-15   2017-06-15  
1        2.0  64.0 2017-06-15   2017-06-15  
2        2.0  51.0 2017-06-16   2017-06-16  
3        2.0  60.0 2017-06-19   2017-06-19  
4        2.0  52.0 2017-06-21   2017-06-21  


# extract Methods data
- number of subjects
- subjects per site
- removed subjects

In [27]:
data101.head(3)

,SubjectCode,SiteID,VisitNm,Q11,Q12,ConsentDate,Age,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10
0,1000.0,1313.0,Baseline,A1784,10.3.2,2017-06-15,63.0,Yes,Yes,Yes,Yes,Yes,Yes,No,No
1,1001.0,1313.0,Baseline,A1778,10.3.2,2017-06-15,64.0,No,Yes,Yes,Yes,Yes,Yes,No,No
2,1002.0,1313.0,Baseline,A1586,10.3.2,2017-06-16,51.0,No,Yes,Yes,Yes,Yes,Yes,No,No


In [ ]:
# data101.groupby(pd.cut(Pxx.index, bins=binedges)).mean().fillna(0)

In [28]:
data101.groupby(by=data101.SubjectCode)

In [34]:
pd.unique(data101.SubjectCode.values)

array([1000., 1001., 1002., 1003., 1004., 1005., 1006., 1007., 1008.,
       1009., 1010., 1011., 1013., 1014., 1015., 1016., 1017., 1018.,
       1019., 1020., 1021., 1022., 1023., 1024., 1025., 1026., 1027.,
       1028., 1029., 1030., 1031., 1032., 1033., 1034., 1035., 1036.,
       1037., 1038., 1039., 1040., 1041., 1042., 1043., 1044., 1045.,
       1046., 1047., 1048., 1049., 1050., 1051., 1052., 1053., 1054.,
       1055., 1056.])

### total number of patients

In [32]:
len(pd.unique(data101.SubjectCode.values))

56

In [33]:
len(data101.SubjectCode.values)

56

### check 4 sites

In [35]:
# checks out
pd.unique(data101.SiteID.values)

array([1313., 1332., 1396., 1018.])

### check n for each site

In [36]:
data101.groupby(by=data101.SiteID)